### Neste código construiremos 3 redes neurais para aproximar derivadas.
### Nos meus testes os resultados foram positivos, mas não posso garantir a consistência dos resultados devido a aleatoriedade do conjunto de dados criado.

In [1]:
import tensorflow as tf
import numpy as np

float_pres='float64'

In [2]:
# Criando conjunto de dados

data_x_list=[]
data_y_list=[]
pi=np.pi

for i in range(50000):
    Δx = 0.01                                 # Distância espacial dos pontos na malha utilizada
    x = tf.range(-2, 2, Δx, dtype=float_pres) # Gerando a malha de pontos no espaço unidimensional
    
    # Gerando uma condição inicial aleatória
    #------------------------------------------------------------------------------------------------------------------
    k1 = tf.random.uniform([1], 0, 20, dtype='int32')   # Amostrando uma frequência aleatória para a função seno
    k1 = tf.cast(k1, dtype=float_pres)                  # Mudando o tipo do tensor
    k2 = tf.random.uniform([1], 0, 20, dtype='int32')   # Amostrando uma frequência aleatória para a função seno
    k2 = tf.cast(k2, dtype=float_pres)                  # Mudando o tipo do tensor
    a  = tf.random.uniform([1], 0, 1, dtype=float_pres) # Amostrando um peso aleatória para ponderar as funções seno
    b  = tf.random.uniform([1], 0, 2, dtype=float_pres) # Amostrando um modificador de amplitude aleatório
    #------------------------------------------------------------------------------------------------------------------
    
    # Valor da função
    u1 =     a * tf.expand_dims(tf.math.sin(k1*pi*x), axis=0) # Gerando pontos de acordo com a primeira função seno
    u2 = (1-a) * tf.expand_dims(tf.math.sin(k2*pi*x), axis=0) # Gerando pontos de acordo com a segunda função seno
    
    # Valor da derivada
    du1= a*k1*pi*tf.expand_dims(tf.math.cos(k1*pi*x), axis=0)
    du2= (1-a)*k2*pi*tf.expand_dims(tf.math.cos(k2*pi*x), axis=0)
    
    u = b*(u1+u2) 
    du= b*(du1+du2)
    
    data_x_list.append(u)
    data_y_list.append(du)

data_x=tf.concat(data_x_list,axis=0)
data_y=tf.concat(data_y_list,axis=0)

train_x=data_x[:40000]
train_y=data_y[:40000]
test_x=data_x[-10000:]
test_y=data_y[-10000:]

In [16]:
# Criando um modelo sequencial
model_foward=tf.keras.models.Sequential()

# Adicionando camadas
model_foward.add(tf.keras.layers.Reshape([400,1])) # A camada de convolução exige uma dimensão extra para a "cor" da imagem,
                                            # por isso o reshape para transformar o tensor de tamanho n x 400 em um tensor de tamanho n x 400 x 1
model_foward.add(tf.keras.layers.Conv1D(filters=1,activation='linear', kernel_size=2, use_bias=False,dtype=float_pres)) # Camada de convolução com ativação linear, após a convolução, a dimensão do vetor será n x 399.
                                                                                                       # Lembrando que, se temos um input com dimensão n x s x r e uma convolução com filtro a x b (a é o tamanho do kernel e b é a quantidade de filtros) 
                                                                                                       # Então o output da convolução será: n x (s-a+1) x b
                                                                                                       # Dada a natureza do problema, sabemos que não é necessário um bias (use_bias=False), então removemos ele para evitar overfitting.
model_foward.add(tf.keras.layers.Flatten()) # Esta camada remove a dimensão extra, transformando um tensor de tamanho n x 399 x 1 em um tensor de tamanho n x 399

optimizer = tf.keras.optimizers.Adam(learning_rate=10**-1, beta_1=0.9, beta_2=0.999, clipnorm=1.0)
model_foward.compile(loss='mean_squared_error',optimizer=optimizer)

In [20]:
history = model_foward.fit(train_x, train_y[:,:-1], # Como estamos calculando a derivada "pra frente", devemos remover a última derivada
                           batch_size=512,
                           epochs=30,
                           validation_split=0.2)
test_scores = model_foward.evaluate(test_x, test_y[:,:-1], verbose=1)
print('Test loss:', test_scores)

Epoch 1/30
63/63 [==============================] - 1s 11ms/step - loss: 13.4183 - val_loss: 13.5986
Epoch 2/30
63/63 [==============================] - 1s 12ms/step - loss: 13.4224 - val_loss: 13.6053
Epoch 3/30
63/63 [==============================] - 1s 12ms/step - loss: 13.4188 - val_loss: 13.6001
Epoch 4/30
63/63 [==============================] - 1s 12ms/step - loss: 13.4279 - val_loss: 13.6134
Epoch 5/30
63/63 [==============================] - 1s 12ms/step - loss: 13.4234 - val_loss: 13.6011
Epoch 6/30
63/63 [==============================] - 1s 12ms/step - loss: 13.4238 - val_loss: 13.6114
Epoch 7/30
63/63 [==============================] - 1s 12ms/step - loss: 13.4198 - val_loss: 13.6005
Epoch 8/30
63/63 [==============================] - 1s 12ms/step - loss: 13.4207 - val_loss: 13.5986
Epoch 9/30
63/63 [==============================] - 1s 12ms/step - loss: 13.4198 - val_loss: 13.5986
Epoch 10/30
63/63 [==============================] - 1s 12ms/step - loss: 13.4195 - val_los

In [21]:
# Pegando os pesos da segunda camada (a de convolução) da rede e multiplicando por Δx para ver se o valor está correto (o ideal seria -1 e 1).
model_foward.layers[1].weights[0]*Δx

<tf.Tensor: shape=(2, 1, 1), dtype=float64, numpy=
array([[[-0.95048655]],

       [[ 1.01217661]]])>

In [6]:
# Criando um modelo sequencial
model_backward=tf.keras.models.Sequential()

# Adicionando camadas
model_backward.add(tf.keras.layers.Reshape([400,1])) # A camada de convolução exige uma dimensão extra para a "cor" da imagem,
                                            # por isso o reshape para transformar o tensor de tamanho n x 400 em um tensor de tamanho n x 400 x 1
model_backward.add(tf.keras.layers.Conv1D(filters=1,activation='linear', kernel_size=2, use_bias=False)) # Camada de convolução com ativação linear, após a convolução, a dimensão do vetor será n x 399.
                                                                                                         # Lembrando que, se temos um input com dimensão n x s x r e uma convolução com filtro a x b (a é o tamanho do kernel e b é a quantidade de filtros) 
                                                                                                         # Então o output da convolução será: n x (s-a+1) x b
model_backward.add(tf.keras.layers.Flatten()) # Esta camada remove a dimensão extra, transformando um tensor de tamanho n x 399 x 1 em um tensor de tamanho n x 399


optimizer = tf.keras.optimizers.Adam(learning_rate=10**-1, beta_1=0.9, beta_2=0.999, clipnorm=1.0)
model_backward.compile(loss='mean_squared_error',optimizer=optimizer,metrics=['mean_squared_error'])

In [7]:
history = model_backward.fit(train_x, train_y[:,1:], # Como estamos calculando a derivada "pra trás", devemos remover a primeira derivada
                    batch_size=512,
                    epochs=30,
                    validation_split=0.2)
test_scores = model_backward.evaluate(test_x, test_y[:,1:], verbose=1)
print('Test loss:', test_scores[0])
print('Test accuracy:', test_scores[1])

Epoch 1/30
63/63 [==============================] - 1s 11ms/step - loss: 514.9156 - mean_squared_error: 514.9156 - val_loss: 488.0776 - val_mean_squared_error: 488.0776
Epoch 2/30
63/63 [==============================] - 0s 7ms/step - loss: 452.8593 - mean_squared_error: 452.8593 - val_loss: 427.2430 - val_mean_squared_error: 427.2430
Epoch 3/30
63/63 [==============================] - 0s 7ms/step - loss: 394.6532 - mean_squared_error: 394.6532 - val_loss: 370.6004 - val_mean_squared_error: 370.6004
Epoch 4/30
63/63 [==============================] - 0s 8ms/step - loss: 340.6747 - mean_squared_error: 340.6747 - val_loss: 318.1131 - val_mean_squared_error: 318.1131
Epoch 5/30
63/63 [==============================] - 0s 8ms/step - loss: 290.8527 - mean_squared_error: 290.8527 - val_loss: 269.8255 - val_mean_squared_error: 269.8255
Epoch 6/30
63/63 [==============================] - 0s 7ms/step - loss: 244.9574 - mean_squared_error: 244.9574 - val_loss: 225.6953 - val_mean_squared_error: 

In [8]:
# Pegando os pesos da segunda camada (a de convolução) da rede e multiplicando por Δx para ver se o valor está correto (o ideal seria -1 e 1).
model_backward.layers[1].weights[0]*Δx

<tf.Tensor: shape=(2, 1, 1), dtype=float32, numpy=
array([[[-1.0128758 ]],

       [[ 0.94938374]]], dtype=float32)>

In [9]:
# Criando um modelo sequencial
model_middle=tf.keras.models.Sequential()

# Adicionando camadas
model_middle.add(tf.keras.layers.Reshape([400,1])) # A camada de convolução exige uma dimensão extra para a "cor" da imagem,
                                            # por isso o reshape para transformar o tensor de tamanho n x 400 em um tensor de tamanho n x 400 x 1
# Observe que, para a derivada centrada, o kernel_size deve ser 3.
model_middle.add(tf.keras.layers.Conv1D(filters=1,activation='linear', kernel_size=3, use_bias=False)) # Camada de convolução com ativação linear, após a convolução, a dimensão do vetor será n x 398.
                                                                                                       # Lembrando que, se temos um input com dimensão n x s x r e uma convolução com filtro a x b (a é o tamanho do kernel e b é a quantidade de filtros) 
                                                                                                       # Então o output da convolução será: n x (s-a+1) x b
model_middle.add(tf.keras.layers.Flatten()) # Esta camada remove a dimensão extra, transformando um tensor de tamanho n x 399 x 1 em um tensor de tamanho n x 399


optimizer = tf.keras.optimizers.Adam(learning_rate=10**-1, beta_1=0.9, beta_2=0.999, clipnorm=1.0)
model_middle.compile(loss='mean_squared_error',optimizer=optimizer,metrics=['mean_squared_error'])

In [10]:
history = model_middle.fit(train_x, train_y[:,1:-1], # Como estamos calculando a derivada centrada, devemos remover a primeira e a última derivada
                    batch_size=512,
                    epochs=30,
                    validation_split=0.2)
test_scores = model_middle.evaluate(test_x, test_y[:,1:-1], verbose=1)
print('Test loss:', test_scores[0])
print('Test accuracy:', test_scores[1])

Epoch 1/30
63/63 [==============================] - 1s 11ms/step - loss: 479.4188 - mean_squared_error: 479.4188 - val_loss: 422.3605 - val_mean_squared_error: 422.3605
Epoch 2/30
63/63 [==============================] - 0s 8ms/step - loss: 363.6899 - mean_squared_error: 363.6899 - val_loss: 313.4422 - val_mean_squared_error: 313.4422
Epoch 3/30
63/63 [==============================] - 0s 8ms/step - loss: 263.7209 - mean_squared_error: 263.7209 - val_loss: 220.7431 - val_mean_squared_error: 220.7431
Epoch 4/30
63/63 [==============================] - 0s 7ms/step - loss: 180.0979 - mean_squared_error: 180.0979 - val_loss: 144.2630 - val_mean_squared_error: 144.2630
Epoch 5/30
63/63 [==============================] - 0s 8ms/step - loss: 112.2075 - mean_squared_error: 112.2075 - val_loss: 84.0017 - val_mean_squared_error: 84.0017
Epoch 6/30
63/63 [==============================] - 0s 8ms/step - loss: 60.5434 - mean_squared_error: 60.5434 - val_loss: 39.9592 - val_mean_squared_error: 39.95

In [11]:
# Pegando os pesos da segunda camada (a de convolução) da rede e multiplicando por Δx para ver se o valor está correto (o ideal seria -0.5, 0 e 0.5).
model_middle.layers[1].weights[0]*Δx

<tf.Tensor: shape=(3, 1, 1), dtype=float32, numpy=
array([[[-5.1925200e-01]],

       [[ 4.1817068e-04]],

       [[ 5.1882267e-01]]], dtype=float32)>

### De maneira geral, os resultados foram satisfatórios, lembrando que o conjunto de dados é relativamente pequeno (50.000 observações), então é natural que a rede não consiga encontrar os pesos exatos devido a algum viés aleatório do conjunto de treino.
### Fica a critério do usuário aumentar o tamanho do conjunto de treino afim de gerar aproximações melhores.